<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.tanh(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1328125
Current benign train loss: 2.2833943367004395

Current batch: 100
Current benign train accuracy: 0.2890625
Current benign train loss: 2.024975061416626

Current batch: 200
Current benign train accuracy: 0.328125
Current benign train loss: 1.7856388092041016

Current batch: 300
Current benign train accuracy: 0.3515625
Current benign train loss: 1.7832753658294678

Total benign train accuarcy: 31.086
Total benign train loss: 733.2722601890564

[ Test epoch: 0 ]

Test accuarcy: 37.5
Test average loss: 0.017249840259552
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.328125
Current benign train loss: 1.805242896080017

Current batch: 100
Current benign train accuracy: 0.328125
Current benign train loss: 1.6565145254135132

Current batch: 200
Current benign train accuracy: 0.453125
Current benign train loss: 1.5089435577392578

Current batch: 300
Current benign train accuracy: 


Current batch: 200
Current benign train accuracy: 0.796875
Current benign train loss: 0.5194969773292542

Current batch: 300
Current benign train accuracy: 0.8203125
Current benign train loss: 0.4722291827201843

Total benign train accuarcy: 78.088
Total benign train loss: 243.17944160103798

[ Test epoch: 13 ]

Test accuarcy: 77.94
Test average loss: 0.006485875633358955
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.7265625
Current benign train loss: 0.7390180826187134

Current batch: 100
Current benign train accuracy: 0.84375
Current benign train loss: 0.4894520342350006

Current batch: 200
Current benign train accuracy: 0.75
Current benign train loss: 0.740328848361969

Current batch: 300
Current benign train accuracy: 0.796875
Current benign train loss: 0.5480163097381592

Total benign train accuarcy: 79.274
Total benign train loss: 233.0738343000412

[ Test epoch: 14 ]

Test accuarcy: 72.56
Test average loss: 0.008099495321512223
Model Saved


Total benign train accuarcy: 84.57
Total benign train loss: 174.52313724160194

[ Test epoch: 26 ]

Test accuarcy: 75.05
Test average loss: 0.00787526285648346
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.8671875
Current benign train loss: 0.44604596495628357

Current batch: 100
Current benign train accuracy: 0.796875
Current benign train loss: 0.6334599852561951

Current batch: 200
Current benign train accuracy: 0.859375
Current benign train loss: 0.5300330519676208

Current batch: 300
Current benign train accuracy: 0.8515625
Current benign train loss: 0.47417744994163513

Total benign train accuarcy: 84.896
Total benign train loss: 170.43591246008873

[ Test epoch: 27 ]

Test accuarcy: 81.32
Test average loss: 0.005457740291953087
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3368857204914093

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign t


Current batch: 0
Current benign train accuracy: 0.859375
Current benign train loss: 0.3627622425556183

Current batch: 100
Current benign train accuracy: 0.8359375
Current benign train loss: 0.48903125524520874

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.28939276933670044

Current batch: 300
Current benign train accuracy: 0.8671875
Current benign train loss: 0.3481972813606262

Total benign train accuarcy: 87.13
Total benign train loss: 146.8813348710537

[ Test epoch: 40 ]

Test accuarcy: 79.85
Test average loss: 0.006129591292142868
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.34512826800346375

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.2975952625274658

Current batch: 200
Current benign train accuracy: 0.875
Current benign train loss: 0.3032955825328827

Current batch: 300
Current benign train accuracy: 0.8671875
Cu


Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.1102360188961029

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16230590641498566

Total benign train accuarcy: 94.15
Total benign train loss: 67.40896800532937

[ Test epoch: 53 ]

Test accuarcy: 90.59
Test average loss: 0.0029449089385569097
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.08241688460111618

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1222381442785263

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.06377443671226501

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.1731192022562027

Total benign train accuarcy: 94.25
Total benign train loss: 64.4266613945365

[ Test epoch: 54 ]

Test accuarcy: 90.74
Test average loss: 0.002913809061795473
Mo


Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.13241499662399292

Total benign train accuarcy: 95.684
Total benign train loss: 47.93812073394656

[ Test epoch: 66 ]

Test accuarcy: 90.35
Test average loss: 0.0032026615008711816
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.06497523933649063

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.12486683577299118

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.061367571353912354

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.14600691199302673

Total benign train accuarcy: 95.792
Total benign train loss: 46.422050669789314

[ Test epoch: 67 ]

Test accuarcy: 90.6
Test average loss: 0.003085464759171009
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign 


Total benign train accuarcy: 96.388
Total benign train loss: 39.99413602426648

[ Test epoch: 79 ]

Test accuarcy: 89.44
Test average loss: 0.0037747831493616106
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.08484923839569092

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.09681805968284607

Current batch: 200
Current benign train accuracy: 0.9609375
Current benign train loss: 0.09465551376342773

Current batch: 300
Current benign train accuracy: 0.9765625
Current benign train loss: 0.10149942338466644

Total benign train accuarcy: 96.416
Total benign train loss: 39.50326347351074

[ Test epoch: 80 ]

Test accuarcy: 90.29
Test average loss: 0.003449175530672073
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.14472244679927826

Current batch: 100
Current benign train accuracy: 0.921875
Current benign 


Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.025838129222393036

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.06301409006118774

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.0862441435456276

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.038569577038288116

Total benign train accuarcy: 98.682
Total benign train loss: 17.17120099812746

[ Test epoch: 93 ]

Test accuarcy: 91.78
Test average loss: 0.003006816103309393
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.02835959382355213

Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07926131784915924

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.0316004678606987

Current batch: 300
Current benign train accuracy: 0.96875
Current b